In [1]:
import cv2
import numpy as np
from osgeo import gdal
import glob
import csv
import pandas as pd

In [2]:
#headers = ['File name','Coordinates of image center','Green percentage']
file_list = glob.glob(r'D:\SHUBHAM\CCBD\QGIS\BLR_CLIP\*.tif')
images_list = []

file_name = 'D:\\SHUBHAM\\CCBD\\information_CLIP.csv'
metafile = 'D:\\SHUBHAM\\CCBD\\statistics_CLIP.csv'
extra_path = 'D:\\SHUBHAM\\CCBD\\QGIS\\BLR_CLIP\\'
area_list = []

In [3]:
for img_path in file_list:
    image_data= []
    img = cv2.imread(img_path)
    h,w,c = img.shape
    ds = gdal.Open(img_path)
    name = img_path.replace(extra_path,'')
    image_data.append(name)
    
    metadata = gdal.Info(ds)
    x = metadata.find("Center")
    location = metadata[x+42:x+70].strip()
    location = location.replace(",","  ")
    location = location.replace('\''," ")
    location = location.replace('\"'," ")
    location = location.replace('d'," ")
    image_data.append(location)
    
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    lower_green = np.array([20,0,0])
    upper_green = np.array([90,245,245])
    mask = cv2.inRange(hsv, lower_green, upper_green)
    white_pixel = np.sum(mask > 0)
    black_pixel = np.sum(mask == 0)
    area = white_pixel/(white_pixel+black_pixel)
    image_data.append(area)
    area_list.append(area)
    
    images_list.append(image_data)
    

In [4]:
area_list.sort()
df = pd.DataFrame({'Green_Area':area_list})
mean = df.Green_Area.mean()
median = df.Green_Area.median()
q25 = df.Green_Area.quantile(0.25)
q75 = df.Green_Area.quantile(0.75)
threshold = df.Green_Area.quantile(0.90)

In [5]:
print(mean,median,q25,q75,threshold)

0.2123845270289186 0.19364136605817103 0.1399248842693685 0.25617957900253296 0.34670276880076867


In [6]:
for data in images_list:
    if(data[2] > threshold):
        print(data)

['BLR_CLIP.1016.tif', '77 43  6.21 E   12 51 23.08 N', 0.3747925583020351]
['BLR_CLIP.1057.tif', '77 28 18.91 E   12 50 22.25 N', 0.36212769674207357]
['BLR_CLIP.1059.tif', '77 29 29.84 E   12 50 21.58 N', 0.4056249454100795]
['BLR_CLIP.112.tif', '77 35 34.94 E   13  7  7.25 N', 0.45375141933793345]
['BLR_CLIP.145.tif', '77 35 34.57 E   13  6 32.46 N', 0.47279238361428944]
['BLR_CLIP.155.tif', '77 41 29.56 E   13  6 28.82 N', 0.387806795353306]
['BLR_CLIP.156.tif', '77 42  5.06 E   13  6 28.44 N', 0.4780330159839287]
['BLR_CLIP.158.tif', '77 43 16.05 E   13  6 27.70 N', 0.5507031181762599]
['BLR_CLIP.159.tif', '77 43 51.55 E   13  6 27.32 N', 0.42440387806795354]
['BLR_CLIP.163.tif', '77 46 13.53 E   13  6 25.80 N', 0.35068564940169444]
['BLR_CLIP.179.tif', '77 36  9.71 E   13  5 57.30 N', 0.37619006026727225]
['BLR_CLIP.187.tif', '77 40 53.69 E   13  5 54.39 N', 0.37959647130753776]
['BLR_CLIP.194.tif', '77 45  2.15 E   13  5 51.77 N', 0.36963926980522316]
['BLR_CLIP.195.tif', '77 45 

In [7]:

# writing to csv file  
with open(metafile, 'w',newline='') as csvfile:  
    # creating a csv writer object  
    csvwriter = csv.writer(csvfile)  
    csvwriter.writerow(['mean','median','25th quantile','75th quantile','threshold'])    
    # writing the data rows  
    csvwriter.writerow([mean,median,q25,q75,threshold])

In [8]:
# writing to csv file  
with open(file_name, 'w',newline='') as csvfile:  
    # creating a csv writer object  
    csvwriter = csv.writer(csvfile)  
        
    # writing the data rows  
    csvwriter.writerows(images_list)